# Large-scale models and simulation methods - Project

## Data:

[INSEE CENSUS](https://www.insee.fr/fr/statistiques/fichier/6544333/RP2019_INDCVIZB_csv.zip)

[IGN](https://data.geopf.fr/telechargement/download/CONTOURS-IRIS/CONTOURS-IRIS_2-1__SHP__FRA_2021-01-01/CONTOURS-IRIS_2-1__SHP__FRA_2021-01-01.7z)

[INSEE AGGREGATED](https://www.insee.fr/fr/statistiques/fichier/6543200/base-ic-evol-struct-pop-2019_csv.zip)

[URSSAF](https://open.urssaf.fr/api/explore/v2.1/catalog/datasets/etablissements-et-effectifs-salaries-au-niveau-commune-x-ape-last/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B)


In [46]:
from pathlib import Path
import pandas as pd
import pandas as pd
import geopandas as gpd
import plotly.express as px
from sys import exit
import matplotlib.pyplot as plt

In [ ]:
DATA_FOLDER = Path("./data/")
CENSUS_DATA = DATA_FOLDER / "FD_INDCVIZB_2019.csv"
IRIS_DATA = DATA_FOLDER / "CONTOURS-IRIS.shp"
POULATION_DATA = DATA_FOLDER / "base-ic-evol-struct-pop-2019.CSV"
EMPLOYMENT_DATA = (
    DATA_FOLDER
    / "etablissements-et-effectifs-salaries-au-niveau-commune-x-ape-last.csv"
)

print("DATA_FOLDER:", DATA_FOLDER)

if not DATA_FOLDER.exists():
    print("Creating data folder")
    DATA_FOLDER.mkdir()
if not CENSUS_DATA.exists():
    print(f"CENSUS data is missing! Please add: {CENSUS_DATA}")
    exit()
if not IRIS_DATA.exists():
    print(f"IRIS data is missing! Please add: {IRIS_DATA}")
    exit()
if not POULATION_DATA.exists():
    print(f"POPULATION data is missing! Please add: {POULATION_DATA}")
    exit()
if not EMPLOYMENT_DATA.exists():
    print(f"EMPLOYMENT data is missing! Please add: {EMPLOYMENT_DATA}")
    exit()

In [ ]:
# Census data

columns = {
    "IRIS": str,
    "IPONDI": float,
    "AGED": int,
    "CS1": int,
    "DEPT": int,
}
df_census = pd.read_csv(CENSUS_DATA, sep=";", dtype=columns, usecols=columns.keys())
df_census = df_census.rename(
    columns={
        "IRIS": "iris_id",
        "IPONDI": "weight",
        "AGED": "age",
        "CS1": "csp",
        "DEPT": "department",
    }
)
df_census = df_census[df_census["department"] == 28]
df_census = df_census.drop(columns=["department"])

df_census["department_id"] = df_census["iris_id"].str[:2]
df_census["municipality_id"] = df_census["iris_id"].str[:5]

municiplaities_in_department = df_census["municipality_id"].unique()

print(df_census.head())
print(df_census.describe())
print(df_census.info())

In [ ]:
# Iris data

df_iris: gpd.GeoDataFrame = gpd.read_file(IRIS_DATA)
df_iris = df_iris.rename(
    columns={
        "INSEE_COM": "municipality_id",
        "CODE_IRIS": "iris_id",
    }
)[["iris_id", "municipality_id", "geometry"]]
df_iris["department_id"] = df_iris["iris_id"].str[:2]

print(df_iris.head())

In [ ]:
# Population data

columns = ["COM", "C19_POP15P"]
columns += ["C19_POP15P_CS{}".format(k) for k in range(1, 9)]

df_population = pd.read_csv(
    POULATION_DATA,
    sep=";",
    usecols=columns,
    dtype={"COM": str},
)

renamed_columns = ["municipality_id", "population"]
renamed_columns += ["csp_{}".format(k) for k in range(1, 9)]
df_population.columns = renamed_columns

df_population = df_population.groupby("municipality_id").sum().reset_index()

df_population = df_population[
    df_population["municipality_id"].isin(municiplaities_in_department)
]

print(df_population.head())
print(df_population.describe())
print(df_population.info())

In [ ]:
# Employment data

df_employment = pd.read_csv(
    EMPLOYMENT_DATA,
    sep=";",
    usecols=["Code commune", "Effectifs salariés 2019"],
    dtype={"Code commune": str},
)

df_employment = df_employment.rename(
    columns={"Code commune": "municipality_id", "Effectifs salariés 2019": "employment"}
)

df_employment = df_employment.groupby("municipality_id").sum().reset_index()

df_employment = df_employment[
    df_employment["municipality_id"].isin(municiplaities_in_department)
]

print(df_employment.head())
print(df_employment.describe())
print(df_employment.info())

## Exercise 1.1: Study area


### (1) How many person observations are included in the census for that department?


In [ ]:
observations = len(df_census)
print(f"Number of observations: {observations}")

### (2) How many persons live in that department?


In [ ]:
people = df_census["weight"].sum()
print(f"Number of people: {people}")

### (3) How many municipalities are there in the department?


In [ ]:
print(f"Municipalities: {municiplaities_in_department}")
print(f"Number of municipalities: {municiplaities_in_department.size}")

### Prepare a map that shows where the department is located inside of France.


In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
df_iris.plot(ax=ax, color="lightgrey", alpha=0.5)
df_iris[df_iris["department_id"] == "28"].plot(
    ax=ax,
    color="blue",
)

ax.set_title("Departement 28 in France")
ax.axis("off")

plt.show()

## Exercise 1.2: Territorial analysis I


### Plot the age distribution of the persons living in the department. Indicate the average age of persons living in the territory.


In [ ]:
weighted_counts = (
    df_census.groupby("age")["weight"]
    .sum()
    .reset_index()
    .rename(columns={"weight": "weighted_count"})
)

weighted_avg_age = (
    weighted_counts["age"] * weighted_counts["weighted_count"]
).sum() / weighted_counts["weighted_count"].sum()

fig = px.bar(
    weighted_counts,
    x="age",
    y="weighted_count",
    title="Age Distribution",
)
fig.add_vline(
    x=weighted_avg_age,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Avg. age: {weighted_avg_age:.2f}",
    annotation_position="top",
)
fig.show()

print(f"Avg. age: {weighted_avg_age:.2f}")

### Make a map of the municipalities in the study area and indicate the average age of their population. Which municipality is the youngest, which one is the oldest?


In [ ]:
spatial_municipalities = df_iris.dissolve(by="municipality_id").reset_index()

df_census_grouped = (
    df_census.groupby("municipality_id")
    .apply(
        lambda x: pd.Series(
            {
                "total_weight": x["weight"].sum(),
                "avg_age": (
                    (x["age"] * x["weight"]).sum() / x["weight"].sum()
                    if x["weight"].sum() > 0
                    else 0
                ),
            }
        )
    )
    .reset_index()
)

spatial_data = spatial_municipalities.merge(
    df_census_grouped, on="municipality_id", how="left"
)

fig, ax = plt.subplots(figsize=(12, 8))
spatial_data.plot(
    column="avg_age",
    cmap="viridis",
    legend=True,
    ax=ax,
    legend_kwds={"label": "Average Age"},
)
ax.set_title("Average Age of Population by Municipality")
ax.set_axis_off()

plt.show()

youngest = spatial_data.loc[spatial_data["avg_age"].idxmin()]
oldest = spatial_data.loc[spatial_data["avg_age"].idxmax()]
print(
    f"Youngest Municipality: ID {youngest['municipality_id']} with Avg Age {youngest['avg_age']:.2f}"
)
print(
    f"Oldest Municipality: ID {oldest['municipality_id']} with Avg Age {oldest['avg_age']:.2f}"
)

### Plot the distribution of socio-professional categories of the overall study area and for at least three individual municipalities. Describe if you see differences between municipalities and the study area in general.


In [ ]:
def plot_csp_distribution(df_census: pd.DataFrame, title: str = "CSP Distribution"):
    df_csp = df_census.groupby("csp")["weight"].sum().reset_index()

    df_csp["csp"] = df_csp["csp"].replace(
        {
            1: "Agriculteurs",
            2: "Artisans",
            3: "Cadres",
            4: "Intermédiaires",
            5: "Employés",
            6: "Ouvriers",
            7: "Retraités",
            8: "Autres",
        }
    )

    fig = px.histogram(df_csp, x="csp", y="weight", title=title).update_xaxes(
        categoryorder="total descending"
    )
    fig.show()


plot_csp_distribution(
    df_census,
    title="CSP Distribution - Departement '28'",
)
for municipality_id in df_census["municipality_id"].unique():
    plot_csp_distribution(
        df_census[df_census["municipality_id"] == municipality_id],
        title=f"CSP Distribution - Municipality '{municipality_id}'",
    )

## Exercise 1.3: Territorial analysis II


### Create a bar plot indicating the number of working inhabitants in each municipality.


In [ ]:
working_csp = [1, 2, 3, 4, 5, 6]
df_working = df_census[df_census["csp"].isin(working_csp)]

working_by_municipality = (
    df_working.groupby("municipality_id")["weight"]
    .sum()
    .reset_index()
    .rename(columns={"weight": "working_population"})
)

fig = px.bar(
    working_by_municipality,
    x="municipality_id",
    y="working_population",
    title="Working Population by Municipality",
).update_xaxes(categoryorder="total descending")
fig.show()

### Create a bar plot indicating the number of employees in each municipality


In [ ]:
fig = px.bar(
    df_employment,
    x="municipality_id",
    y="employment",
    title="Number of Employees by Municipality",
).update_xaxes(categoryorder="total descending")
fig.show()

### Make a map of the study area which, for each municipality, shows the difference between working inhabitants and employees. Which municipality the largest net outflow (inhabitants - employees), which one the largest net inflow of employees?


In [ ]:
df_combined = df_employment.merge(working_by_municipality, on="municipality_id")
df_combined["net_difference"] = (
    df_combined["working_population"] - df_combined["employment"]
)

spatial_data = spatial_municipalities.merge(
    df_combined, on="municipality_id", how="left"
)
fig, ax = plt.subplots(figsize=(12, 8))
spatial_data.plot(
    column="net_difference",
    cmap="coolwarm",
    legend=True,
    ax=ax,
    legend_kwds={"label": "Net Difference"},
)
ax.set_title("Net Difference of Working Population and Employees by Municipality")
ax.set_axis_off()

plt.show()

largest_deficit = spatial_data.loc[spatial_data["net_difference"].idxmin()]
largest_surplus = spatial_data.loc[spatial_data["net_difference"].idxmax()]

print(
    f"Largest Deficit Municipality: ID {largest_deficit['municipality_id']} with Net Difference {largest_deficit['net_difference']}"
)
print(
    f"Largest Surplus Municipality: ID {largest_surplus['municipality_id']} with Net Difference {largest_surplus['net_difference']}"
)